In [1]:
# 공동주택가격정보 자료(like apt, yeonlip)
# 공동주택가격정보 자료(like dandok)
# http://www.nsdi.go.kr/lxportal/?menuno=2679
# made by Beans Im

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import glob
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [3]:
pbar = ProgressBar()
pbar.register()
date = '2021-12-29'

In [4]:
paths = glob.glob('D:/feature_data/개별주택가격정보/*.csv')
# ['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '건축물대장고유번호', '기준연도',
# '기준월', '동코드', '동명', '토지대장면적', '산정대지면적', '건물전체연면적', '건물산정연면적', '주택가격',
# '표준지여부', '데이터기준일자']

In [5]:
dandok = pd.DataFrame()
for path in paths:
    print(path)
    tmp = dd.read_csv(
        path, 
        usecols=['법정동명', '지번', '기준연도', '기준월', '주택가격'],
        dtype={'지번': 'str', '기준연도': 'str', '기준월': 'str'},
        encoding = 'cp949'
    )
    tmp = tmp[(tmp['기준연도'].isin(['2021']))].compute()
    dandok = pd.concat([dandok, tmp], ignore_index=True)
    
dandok.drop_duplicates(inplace = True)
dandok.rename(columns = {'법정동명' : 'dong_address', 
                         '지번' : 'bunji', 
                         '기준연도' : 'base_year', 
                         '기준월' : 'base_month',
                         '주택가격' : 'gov_price'}, inplace = True)   
dandok = dandok[dandok['base_month'] == '01'].reset_index(drop=True)
dandok['address_dong'] = dandok['dong_address'] + ' ' + dandok['bunji'] + '번지'
dandok['address_dong'] = dandok['address_dong'].str.strip()
dandok = dandok.drop(['dong_address', 'bunji', 'base_month'], axis=1)
print(dandok.shape)

D:/feature_data/개별주택가격정보\AL_11_D169_20210714.csv
[########################################] | 100% Completed |  9.9s
D:/feature_data/개별주택가격정보\AL_26_D169_20210714.csv
[########################################] | 100% Completed |  5.1s
D:/feature_data/개별주택가격정보\AL_27_D169_20210714.csv
[########################################] | 100% Completed |  4.3s
D:/feature_data/개별주택가격정보\AL_28_D169_20210714.csv
[########################################] | 100% Completed |  2.7s
D:/feature_data/개별주택가격정보\AL_29_D169_20210714.csv
[########################################] | 100% Completed |  2.3s
D:/feature_data/개별주택가격정보\AL_30_D169_20210714.csv
[########################################] | 100% Completed |  2.2s
D:/feature_data/개별주택가격정보\AL_31_D169_20210714.csv
[########################################] | 100% Completed |  1.8s
D:/feature_data/개별주택가격정보\AL_36_D169_20210714.csv
[########################################] | 100% Completed |  0.6s
D:/feature_data/개별주택가격정보\AL_41_D169_20210714.csv
[##############

In [6]:
paths = glob.glob('D:/feature_data/공동주택가격정보/*.csv')
# ['고유번호', '법정동코드', '법정동명', '특수지구분코드', '특수지구분명', '지번', '기준연도', '기준월',
# '공동주택코드', '공동주택구분코드', '공동주택구분명', '특수지명', '공동주택명', '동명', '층명', '호명',
# '전용면적', '공시가격', '데이터기준일자'],

In [7]:
public = pd.DataFrame()
for path in paths:
    print(path)
    tmp = dd.read_csv(
        path, 
        usecols=['법정동명', '지번', '기준연도', '기준월', '공동주택구분명', '동명', '층명', '호명', '공시가격'],
        dtype={'동명': 'object',
               '지번': 'object',
               '호명': 'object',
               '공시가격': 'float64',
               '기준연도': 'object',
               '기준월': 'float64',
               '층명': 'float64'},
        encoding = 'cp949'
    )
    tmp = tmp[(tmp['기준연도'].isin(['2021']))].compute()
    public = pd.concat([public, tmp], ignore_index=True)
    
public.drop_duplicates(inplace = True)

D:/feature_data/공동주택가격정보\AL_11_D167_20210817.csv
[########################################] | 100% Completed | 45.9s
D:/feature_data/공동주택가격정보\AL_26_D167_20210817.csv
[########################################] | 100% Completed | 18.7s
D:/feature_data/공동주택가격정보\AL_27_D167_20210817.csv
[########################################] | 100% Completed | 12.7s
D:/feature_data/공동주택가격정보\AL_28_D167_20210817.csv
[########################################] | 100% Completed | 17.1s
D:/feature_data/공동주택가격정보\AL_29_D167_20210817.csv
[########################################] | 100% Completed |  9.1s
D:/feature_data/공동주택가격정보\AL_30_D167_20210817.csv
[########################################] | 100% Completed |  8.9s
D:/feature_data/공동주택가격정보\AL_31_D167_20210817.csv
[########################################] | 100% Completed |  6.7s
D:/feature_data/공동주택가격정보\AL_36_D167_20210817.csv
[########################################] | 100% Completed |  1.3s
D:/feature_data/공동주택가격정보\AL_41_D167_20210817.csv
[##############

In [8]:
public.rename(columns = {
    '법정동명' : 'dong_address', 
    '지번' : 'bunji', 
    '기준연도' : 'base_year', 
    '기준월' : 'base_month', 
    '공동주택구분명' : 'type', 
    '동명' : 'building_name', 
    '층명' : 'floor_name', 
    '호명' : 'unit_name', 
    '공시가격' : 'gov_price',
}, inplace = True)

In [9]:
public = public.reset_index(drop=True)
public['address_dong'] = public['dong_address'] + ' ' + public['bunji'] + '번지'
public['address_dong'] = public['address_dong'].str.strip()
public = public.drop(['dong_address', 'bunji', 'base_month'], axis=1)
print(public.shape)

(14052025, 7)


In [10]:
# This one change floor name to building name
change = public[public['address_dong'] == '서울특별시 종로구 청운동 1번지'].index
public.loc[change, 'floor_name'] = public.loc[change, 'building_name']
public.loc['building_name', change] = np.nan

In [11]:
dandok.to_pickle('gov_price_dandok_' + date + '.pkl.gz', compression = 'gzip', protocol = 4)
public.to_pickle('gov_price_cohousing_' + date + '.pkl.gz', compression = 'gzip', protocol = 4)